## Assumptions
* space_dependent - these are cast to functions, not commutative

* momentum_symbols - will be specified (default to kx, ky, kz probably) and not commutative

* constant - commutative, everything else

In [1]:
import sympy

from sympy.interactive import printing
printing.init_printing(use_latex='mathjax')

In [2]:
coord = sympy.symbols('x y z', commutative=False)

In [3]:
def substitute_functions(expression, space_dependent=[]):
    symbols = [s for s in expr.atoms(sympy.Symbol) if s in space_dependent]
    subs = {s: sympy.Function(s.name)(*coord) for s in symbols}
    
    return expression.subs(subs)

### Defining input expression

In [4]:
A, B, C, D = sympy.symbols('A B C D', commutative=False)
space_dependent = ['A', 'B']

In [5]:
expr = A*B*C + D; expr

A⋅B⋅C + D

In [6]:
substitute_functions(expr, space_dependent)

A⋅B⋅C + D

# Manipulating and expanding

In [11]:
x, y, z = sympy.symbols('x y z', commutative=False)
a = sympy.symbols('a')
f = sympy.Function('f')
A = sympy.Function('A')

expr1 = (A(x+a) - A(x-a))
expr2 =  (f(x+a) - 2*f(x) + f(x-a)) 
expr1, expr2

(-A(-a + x) + A(a + x), -2⋅f(x) + f(-a + x) + f(a + x))

In [12]:
expr = expr1 * expr2
expr

(-A(-a + x) + A(a + x))⋅(-2⋅f(x) + f(-a + x) + f(a + x))

In [13]:
expr.expand()

2⋅A(-a + x)⋅f(x) - A(-a + x)⋅f(-a + x) - A(-a + x)⋅f(a + x) - 2⋅A(a + x)⋅f(x) 
+ A(a + x)⋅f(-a + x) + A(a + x)⋅f(a + x)

# Incrementing argument

In [14]:
def increment(f):
    return f.subs(x, x+a)

In [15]:
expr1

-A(-a + x) + A(a + x)

In [16]:
increment(expr1)

-A(x) + A(2⋅a + x)